<a href="https://colab.research.google.com/github/dtoralg/TheValley-Intro_unsupervised/blob/main/%5B01%5D%20-%20Intro_No_Supervisados/%5B01%5D%20-%20Notebooks/E3_Principal_Component_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio de Análisis de Componentes Principales (PCA): Análisis del Dataset IRIS


## Introducción


En este ejercicio vamos a realizar un Análisis de Componentes Principales (PCA), donde aplicaremos técnicas de reducción dimensional para analizar y visualizar el famoso dataset IRIS. PCA es una técnica muy potente que nos permite transformar un conjunto de variables posiblemente correlacionadas en un conjunto más pequeño de variables no correlacionadas, llamadas componentes principales.

Esta técnica es lineal, por lo que solamente capturará relaciones lineales entre variables. En PCA buscamos encontrar las componentes principales (ejes) que consiguen que la covarianza entre dos variables tienda a cero.

Estas covarianzas se pueden ordenar de mayor a menor, con lo cual podremos encontrar aquellos ejes que explican la mayor parte de la varianza del dataset, pudiendo prescindir de aquellos ejes que representan un porcentaje menor de la misma - aplicando así una reducción dimensional.

### Objetivos del Ejercicio

- **Comprender los conceptos básicos del PCA** y su utilidad en el análisis de datos.
- **Preprocesar los datos** del dataset IRIS para prepararlos adecuadamente para el análisis.
- **Aplicar PCA** para reducir la dimensionalidad de los datos del IRIS.
- **Visualizar los resultados de PCA** en 2D para interpretar las relaciones entre las muestras.

### Descripción del Dataset IRIS

El dataset IRIS es uno de los conjuntos de datos más conocidos y utilizados en el campo del Machine Learning y la estadística. Contiene 150 muestras de flores, cada una descrita por cuatro características métricas:

- **Sepal Length (cm):** Longitud del sépalo
- **Sepal Width (cm):** Ancho del sépalo
- **Petal Length (cm):** Longitud del pétalo
- **Petal Width (cm):** Ancho del pétalo

Además, cada muestra está etiquetada con una de las tres especies de iris:

- **Iris-setosa**
- **Iris-versicolor**
- **Iris-virginica**

En este ejercicio, utilizaremos las características métricas para realizar PCA y analizaremos cómo las nuevas componentes principales nos ayudan a entender y visualizar mejor los datos.

### Estructura del Notebook

1. **Importación de Librerías y Carga de Datos:** Preparar el entorno de trabajo e importar el dataset IRIS.
2. **Exploración de Datos:** Analizar las características principales del dataset IRIS.
3. **Preprocesamiento de Datos:** Normalizar los datos para asegurar que todas las características contribuyan equitativamente al análisis.
4. **Aplicación de PCA:** Implementar PCA para reducir la dimensionalidad de los datos.
5. **Visualización de Resultados:** Graficar las componentes principales en 2D para interpretar los clusters formados.

### Requisitos

Para seguir este notebook, asegúrate de tener instaladas las siguientes librerías de Python:

- `numpy`
- `pandas`
- `matplotlib`
- `seaborn`
- `scikit-learn`

Puedes instalarlas utilizando el siguiente comando:

```bash
pip install numpy pandas matplotlib seaborn scikit-learn


## Ejercicio de PCA

Cargamos los datos del dataset iris para seguir con nuestro análisis.

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import plotly.express as px
import numpy as np

In [10]:
df = px.data.iris() # Otra forma de acceder a IRIS, con plotly express
features = ["sepal_width", "sepal_length", "petal_width", "petal_length"] # Marcamos las variables numéricas

fig = px.scatter_matrix(
    df,
    dimensions=features,
    color="species"
)

fig.show()

Extraemos el modelo PCA de los datos de IRIS y hallamos las componentes principales para almacenarlas en `components`

In [11]:
df_norm = StandardScaler().fit_transform(df[features].values)
pca = PCA()
components = pca.fit_transform(df_norm)

Con la siguiente función graficamos los distintos datapoints respecto a las combinaciones lineales de los 4 ejes principales que hemos obtenido.

Recordemos que como en el dataset IRIS tenemos 4 variables, el número máximo de Componentes Principales que podemos obtener siempre será 4.

In [12]:
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(4),
    color=df["species"]
)

fig.show()

Ahora debemos evaluar cual de nuestras 4 componentes principales explica la mayor parte de la varianza. Para ello podemos utilizar la función `explained_variance_ratio_`.

In [13]:
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

fig = px.area(
              x=range(1, exp_var_cumul.shape[0] + 1),
              y=exp_var_cumul
)

fig.update_layout(xaxis_type='category')
fig.show()

Se aprecia que con 1 eje podemos explicar aprox. 70% de la varianza y con 2 ejes alcanzamos prácticamente el 90% de la misma.

Con el método PCA podemos fijar `n_components` para definir el número de ejes principales con los que nos quedamos. En este caso tomamos n = 2 para llegar al 90% de la varianza como hemos visto en el gráfico anterior.

In [14]:
pca = PCA(n_components=2)
components = pca.fit_transform(df_norm)

fig = px.scatter(components,
                 x=0,
                 y=1,
                 color=df['species'])
fig.show()

Finalmente podemos combinar todo de manera gráfica en un solo scatterplot: tanto los datapoints como las componentes principales que hemos seleccionado. Para ello dibujaremos los autovectores con un autovalor unitario definido por la raiz del mismo.

$loading = autovector * \sqrt autovalor$

In [15]:

loadings = pca.components_.T * np.sqrt(pca.explained_variance_) # Calculamos los loadings

fig = px.scatter(components, x=0, y=1, color=df['species']) # Graficamos los datapoints

for i, feature in enumerate(features):
    fig.add_annotation(
        ax=0, ay=0,
        axref="x", ayref="y",
        x=loadings[i, 0],
        y=loadings[i, 1],
        showarrow=True,
        arrowsize=2,
        arrowhead=2,
        xanchor="right",
        yanchor="top"
    )
    fig.add_annotation(
        x=loadings[i, 0],
        y=loadings[i, 1],
        ax=0, ay=0,
        xanchor="center",
        yanchor="bottom",
        text=feature,
        yshift=5,
    )
fig.show()

Extra: Visualización de 3 Componentes Principales en 3D

In [16]:
pca = PCA(n_components=3)
components = pca.fit_transform(df_norm)

total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    components, x=0, y=1, z=2, color=df['species'],
    title=f'Total Explained Variance: {total_var:.2f}%',
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.show()